# Golden Gate Cloning

Imports:

In [6]:
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design
from Bio.Restriction import *
from pydna.seqrecord import SeqRecord
from pydna.amplify import pcr
import goldenhinges
from goldenhinges import OverhangsSelector
import networkx as nx
from itertools import permutations, combinations
from pydna.seqrecord import SeqRecord


rb_iis = RestrictionBatch([AcuI, AlwI, BaeI, BbsI, BbvI, BccI, BceAI, BcgI, BciVI, BcoDI, BfuAI, BmrI, BpmI, BpuEI, BsaI, BsaXI, BseRI, BsgI, BsmAI, BsmBI, BsmFI, BsmI, BspCNI, BspMI, BspQI, BsrDI, BsrI, BtgZI, BtsCI, BtsI, BtsIMutI, CspCI, EarI, EciI, Esp3I, FauI, FokI, HgaI, HphI, HpyAV, MboII, MlyI, MmeI, MnlI, NmeAIII, PaqCI, PleI, SapI, SfaNI])

def _list_sticky_ends_(seqs):
    sticky_ends_dict = {} # com dicionario dá para ver sticky ends de cada sequencia (ou blunt)
    sticky_ends = [] # só para registar diferentes tipos de stiky ends (sem repetir e não por ordem, nem blunts)
    for sequence in seqs:
        end_5 = sequence.seq.five_prime_end()
        end_3 = sequence.seq.three_prime_end()
        sticky_ends_dict[seqs.index(sequence)] = []
        if end_5[0] == 'blunt':
            sticky_ends_dict[seqs.index(sequence)].append((end_5[0])) # blunt
        else:
            sticky_ends_dict[seqs.index(sequence)].append((end_5[1])) # overhang
            sticky_ends.append(end_5[1].upper()) 
        if end_3[0] == 'blunt':
            sticky_ends_dict[seqs.index(sequence)].append((end_3[0])) # blunt
        else:
            sticky_ends_dict[seqs.index(sequence)].append((end_3[1])) # overhang
            sticky_ends.append(end_3[1].upper())

    sticky_ends = list(set(sticky_ends))

    return sticky_ends_dict, sticky_ends


def _compatible_enzyme_(seqs, enzs):
    
    comp_enzymes = enzs.copy()
    
    for e in enzs:
        if e not in rb_iis: # only type IIs restriction enzymes
            comp_enzymes.remove(e)
        else:
            for seq in seqs:
                if e.search(seq.seq) != []:
                    comp_enzymes.remove(e)
                    break

    if comp_enzymes == []:
        raise ValueError('''No type IIs restriction enzymes available on this list
                        that do not cut within the sequences''')
    
    return comp_enzymes


def _design_seqs_(seqs, sticky_ends_dict, compatible_enzymes, circular, sticky_ends):

    def _design_primers_(sequence, compatible_enzymes, f_ovhg, r_ovhg, ind):

        if isinstance(sequence, Dseqrecord):
            sequence = primer_design(sequence)

        golden_hinges = OverhangsSelector(
                    gc_min=0.25,
                    gc_max=0.5,
                    differences=2,
                    forbidden_overhangs= sticky_ends)
        
        overhangs = golden_hinges.generate_overhangs_set(n_overhangs=2)
        
        if f_ovhg == 'blunt':
            overhang_f = overhangs[0]
            sticky_ends.append(overhang_f)
        else:
            overhang_f = SeqRecord(f_ovhg).reverse_complement().seq
        if r_ovhg == 'blunt':
            overhang_r = overhangs[1]
            sticky_ends.append(overhang_r)   
        else:
            overhang_r = SeqRecord(r_ovhg).reverse_complement().seq

        fp = compatible_enzymes[0].site + 'a' + overhang_f + sequence.forward_primer
        rp = compatible_enzymes[0].site + 'a' + overhang_r + sequence.reverse_primer

        pcr_prod = pcr(fp, rp, sequence)
        new_seq = Dseqrecord(pcr_prod)

        sticky_ends_dict[ind][0] = overhang_f
        sticky_ends_dict[ind][1] = overhang_r

        return new_seq

    new_seqs = []

    for ind in range(len(seqs)): # index da sequencia na lista (até ao penultimo)

        if ind == 0:
            if circular:
                previous_sticky_end = sticky_ends_dict[len(seqs)-1][1]
            else:
                previous_sticky_end = 'blunt' # Desta forma está a criar primer com sticky end "aleatório" a não corresponder à ultima seq // Algternativa é mudar para None e depois alterar na função em que não cria um primer para essa ponta mas depois a DNA ligase poderá ligar as duas sequencias
        else:
            previous_sticky_end = sticky_ends_dict[ind-1][1]

        if ind == len(seqs)-1:
            if circular:
                next_sticky_end = sticky_ends_dict[0][0]
            else:
                next_sticky_end = 'blunt' # Desta forma está a criar primer com sticky end "aleatório" a não corresponder à primeira seq
        else:
            next_sticky_end = sticky_ends_dict[ind+1][0] 


        if sticky_ends_dict[ind][0] == 'blunt' and sticky_ends_dict[ind][1] == 'blunt':
            new_seq = _design_primers_(seqs[ind], compatible_enzymes = compatible_enzymes, f_ovhg = previous_sticky_end, r_ovhg = next_sticky_end, ind=ind)

            new_seqs.append(new_seq)

        elif sticky_ends_dict[ind][0] != 'blunt' and sticky_ends_dict[ind][1] != 'blunt':
            new_seqs.append(seqs[ind]) # não fazer nada porque já tem sticky ends (não é preciso enzima porque o design dos outros fragmentos vão coincidir)
    
        else:
            raise ValueError('ERRO DESIGN PRIMER')

    return new_seqs


def GoldenGateDesigner(seqs: list, enzs: list, circular: bool = True) -> list:
    '''
    GoldenGateDesigner is a function that receives a list of sequences and a list of enzymes and, using these, it must return a list of sequences with compatible primers or overhangs.

    Parameters
    ----------
    seqs : list
        list of sequences (order is important)
    enzs : list
        list of enzymes (order is important)
    circular : bool, optional 
        To define if intended final sequence is circular or not, by default True

    Returns
    -------
    new_seqs: list
        list with Dseqrecords and amplicons
    '''

    # STEP 1:  list all sticky ends
    sticky_ends_dict,  sticky_ends = _list_sticky_ends_(seqs)
   

    # STEP 2:  compatible enzymes (enzymes that do not cut within the sequences)
    compatible_enzymes = _compatible_enzyme_(seqs, enzs) 

    # STEP 3: for loop over sequences
    # design relevant primer if needed (add enzyme and comp sticky end)
    new_seqs = _design_seqs_(seqs, sticky_ends_dict, compatible_enzymes, circular, sticky_ends)
   
    # STEP 4: Return a list with Dseqrecords and amplicons

    return new_seqs , compatible_enzymes[0] # o user tem de saber qual a enzima compativel




def _graph_assembly_(list_seqs: list):
    '''
    graph_assembly creates a graph with all the possibilities of assembly between the sequences on the list

    Parameters
    ----------
    list_seqs : list
        list of the sorted sequences with sticky ends for assembly (order is important)

    Returns
    -------
    _type_
        graph
    '''
    G = nx.DiGraph()
    
    for comb in permutations(list_seqs, 2):
        seq1, seq2 = comb
        try: 
            seq1 + seq2
        except:
            continue
        else:
            node1 = list_seqs.index(seq1)+1
            node2 = list_seqs.index(seq2)+1
            G.add_node(node1, dseq=seq1)
            G.add_node(node2, dseq=seq2)
            G.add_edge(node1, node2)
        
    if G.nodes:
        return G
    else:
        raise ValueError('Assembly not possible with these sequences')
    

def _find_all_paths_(graph):
    '''
    find_all_paths is a function that returns a list of all the paths in a graph

    Parameters
    ----------
    graph : _type_
        a graph with multiple nodes and edges (it can be directed or not)

    Returns
    -------
    all_paths : list
        a list of all the possible paths in the graph
    '''
    def _find_paths_(start, end, path=[]):
        '''
        find_paths  is a secondary function for recursive use in all the nodes of the graph

        Parameters
        ----------
        start : _type_
            node of a graph to start the path
        end : _type_
            node of a graph that ends the path
        path : list, optional
            list of the current path when searching, by default []

        Returns
        -------
        paths : list
            list of all the nodes included in the current path
        '''
        path = path + [start]
        if start == end and len(path) > 1:
            return [path]
        if start not in graph:
            return []
        paths = []
        for node in graph[start]:
            if node not in path or (node == end): 
                newpaths = _find_paths_(node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths

    all_paths = []
    for start in graph:
        for end in graph:
            if start != end:
                paths = _find_paths_(start, end)
                all_paths.extend(paths)

        # Add circular paths
        circular_paths = _find_paths_(start, start)
        all_paths.extend(circular_paths)

    return all_paths



def _find_paths_seqs_(paths, grafo):
    '''
    find_paths_seqs is a function that returns a dictionary, the keys are all the paths in a graph and the values are the sequences assembled (for each path) 

    Parameters
    ----------
    paths : list
        list of lists of paths (sequences that can be assembled by that specific order)
    grafo : _type_
        graph that contains all the nodes and edges (sequences and possible assemblies between them)

    Returns
    -------
    sequencias : dict
        a dictionary, the keys are all the paths in a graph and the values are the sequences assembled (for each path) 
    '''
    sequencias = {}
    for path in paths:
        soma = None
        circular = False # bool
        if len(set(path)) != len(path):
            circular = True
            path.pop()
        for i, seq in enumerate(path): # se for circular não se pode repetir para juntar as sequencias 
            if i >= len(list(set(path))): # quando chega ao fim da lista
                break
            if soma is None: # quando está no inicio do path
                soma = grafo.nodes[seq]['dseq']
            else: # continua no path e ir somando à sequencia existente
                soma += grafo.nodes[seq]['dseq']

        if circular:
            soma = Dseqrecord(soma, circular=True)

        sequencias[(tuple(path))] = soma

    # complementar com calculadora seguid para eliminar sequencias repetidas (alguns paths acabam por ser iguais, ciruclares) 
    #### AQUI: retorna 4 seqs (sequencias unicas)
    dicio = sequencias.copy()

    for comb in combinations(sequencias.items(), 2):
        if comb[0][1].circular and comb[1][1].circular:
            if comb[0][1].cseguid() == comb[1][1].cseguid():
                if comb[0][0] in dicio:
                    dicio.pop(comb[0][0])
        elif comb[0][1].useguid() == comb[1][1].useguid(): 
            if comb[0][0] in dicio:
                dicio.pop(comb[0][0])

    return dicio




def GoldenGateAssembler(seqs: list) -> dict:
    '''
    GoldenGateAssembler is a function that receives a list of sequences (fragments) and tries all the possible assembles between them, returns a dictionary of all the possible sequences ligated (with only DNA ligase).

    Parameters
    ----------
    seqs : list
        list of sequences (fragments of DNA)

    Returns
    -------
    dict_seqs : dict
        dictionary with the possible sequences ligated (keys are the order of the sequences ligated; values are the dseqrecords of the sequences assembled)
    '''

    # STEP 1: creates a graph where each node is a sequence and each edge represents the link between sequences (each possible combination in a list of sequences)
    graph = _graph_assembly_(seqs)
    
    # STEP 2: find all paths (each combination of a possible assembled sequence)  
    paths = _find_all_paths_(graph)

    # STEP 3: using the paths as keys, returns all the assembled sequences for each of them (without repetitions) 
    dict_seqs = _find_paths_seqs_(paths, graph)
    
    return dict_seqs

### Example 1 (article)

Creating a list of sequences with Class Dseqrecord:

In [7]:
seq1 = Dseqrecord(
    "ccagttgctaacccttttcgatttcgaaacttacgatg")
seq2 = primer_design(Dseqrecord(
    "ccagttgctaaccttcttcgttcgaaacttacgatg"))

seqs = [seq1, seq2]

for seq in seqs:
    print(seq.figure())
    print()

Dseqrecord(-38)
ccagttgctaacccttttcgatttcgaaacttacgatg
ggtcaacgattgggaaaagctaaagctttgaatgctac

5ccagttgctaaccttct...tcgttcgaaacttacgatg3
                     |||||||||||||||||||
                    3agcaagctttgaatgctac5
5ccagttgctaaccttct3
 |||||||||||||||||
3ggtcaacgattggaaga...agcaagctttgaatgctac5



Creating a list of enzymes:

In [8]:
enzs = [EcoRI, BsaI, BsmBI, BamHI, BpmI]

With these lists, it is possible to see the **GoldenGateDesigner** function in action.

This function returns a new list with the sequences with primers (containing a BsaI site that produce specific sticky ends).

In [9]:
gg_designer = GoldenGateDesigner(seqs, enzs)
print(f'List of the new sequences: {gg_designer[0]}')
for seq in gg_designer[0]:
    print(seq.figure())
    print()

print(f'Compatible enzyme: {gg_designer[1]}')

List of the new sequences: [Dseqrecord(-60), Dseqrecord(-58)]
Dseqrecord(-60)
GGTCTCaAAAGccagttgctaacccttttcgatttcgaaacttacgatgACTTtGAGACC
CCAGAGtTTTCggtcaacgattgggaaaagctaaagctttgaatgctacTGAAaCTCTGG

Dseqrecord(-58)
GGTCTCaACTTccagttgctaaccttcttcgttcgaaacttacgatgAAAGtGAGACC
CCAGAGtTGAAggtcaacgattggaagaagcaagctttgaatgctacTTTCaCTCTGG

Compatible enzyme: BsaI


After designing the sequences, the user can use them for enzymatic restriction. This can be made with this code:


In [10]:
new_list = []

for seq in gg_designer[0]:
    if seq.cut(gg_designer[1]):
        seq1, seq2, seq3 = seq.cut(gg_designer[1]) # gg_designer[1] is the selected compatible enzyme BsaI
        new_list.append(seq2)
        print(seq2.figure())
    else:
        new_list.append(seq)
        print(seq.figure())
    print()

Dseqrecord(-46)
AAAGccagttgctaacccttttcgatttcgaaacttacgatg    
    ggtcaacgattgggaaaagctaaagctttgaatgctacTGAA

Dseqrecord(-44)
ACTTccagttgctaaccttcttcgttcgaaacttacgatg    
    ggtcaacgattggaagaagcaagctttgaatgctacTTTC



Alternative:

In [11]:
_, seq_1, __ = gg_designer[0][0].cut(BsaI)
_, seq_2, __ = gg_designer[0][1].cut(BsaI)

print(seq_1.figure())
print()
print(seq_2.figure())

Dseqrecord(-46)
AAAGccagttgctaacccttttcgatttcgaaacttacgatg    
    ggtcaacgattgggaaaagctaaagctttgaatgctacTGAA

Dseqrecord(-44)
ACTTccagttgctaaccttcttcgttcgaaacttacgatg    
    ggtcaacgattggaagaagcaagctttgaatgctacTTTC


After enzymatic restriction, it is possible to see the **GoldenGateAssembler** function in action.

This function returns a dictionary with all possible combinations of the resulting sequences (possible assembled sequences).

In [12]:
gg_assembler = GoldenGateAssembler(new_list)
print(gg_assembler)

{(2, 1): Dseqrecord(o82)}


In this case (only two sequences with good compatible sticky ends), it shows only one result, that corresponds to a circular sequence of 82 nucleotides.

In [13]:
print(gg_assembler[(2,1)].figure())

Dseqrecord(o82)
CTTTccagttgctaaccttcttcgttcgaaacttacgatgAATCccagttgctaacccttttcgatttcgaaacttacgatg
GAAAggtcaacgattggaagaagcaagctttgaatgctacTTAGggtcaacgattgggaaaagctaaagctttgaatgctac


### Example 2

Creating a list of sequences (after restriction to simulate what happens with sequences already with sticky ends -- not needing primers).

In [43]:
a = Dseqrecord("GGTCTCatgccctaaccccccctaacccacGAGACC")
b = Dseqrecord("GGTCTCacccagtaaccatgtagtaaaaaacccacGAGACC")
c = Dseqrecord("GGTCTCacccacgggggggggggcagtacagtaGAGACC")

list_enz = [EcoRI, BsaI, BsmBI, BamHI, BpmI]

In [44]:
a1,a2,a3  = a.cut(BsaI)
b1,b2,b3 = b.cut(BsaI)
c1,c2,c3 = c.cut(BsaI)

list_seqs = [a2,b2,c2]

**Important Note:** Seeing the figures, it is possible to see that sequence **b** have a common sticky end with sequence **a** (`gggt`) and with sequence **c** (`ccca`).

It will be important to see ahead in the Assembly phase.

In [45]:
for i in list_seqs:
    print(i.figure())
    print()

Dseqrecord(-22)
tgccctaaccccccctaa    
    gattgggggggattgggt

Dseqrecord(-27)
cccagtaaccatgtagtaaaaaa    
    cattggtacatcattttttgggt

Dseqrecord(-25)
cccacgggggggggggcagta    
    gcccccccccccgtcatgtca



GoldenGateDesigner function -- in here it is possible to see that the sequences don't need changes (not neccessary to create primers).

In [46]:
gg_designer = GoldenGateDesigner(list_seqs, list_enz)
print(gg_designer)
print()

for seq in gg_designer[0]:
    print(seq.figure())
    print()

([Dseqrecord(-22), Dseqrecord(-27), Dseqrecord(-25)], BsaI)

Dseqrecord(-22)
tgccctaaccccccctaa    
    gattgggggggattgggt

Dseqrecord(-27)
cccagtaaccatgtagtaaaaaa    
    cattggtacatcattttttgggt

Dseqrecord(-25)
cccacgggggggggggcagta    
    gcccccccccccgtcatgtca



GoldenGateAssembler function -- in here it is possible to see that the objective sequence is present -- (1,2,3): Dseqrecord(-66), it can also show that other sequences can be assembled instead: (1,3): Dseqrecord(-43).

With this example, it is possible to conclude that this experiment has a bad design and will cause problemas when trying to clone this product.

In [47]:
gg_assembler = GoldenGateAssembler(gg_designer[0])
print(gg_assembler)

{(1, 2): Dseqrecord(-45), (1, 2, 3): Dseqrecord(-66), (1, 3): Dseqrecord(-43), (2, 3): Dseqrecord(-48)}


### Example 3

Creating a list of sequences and a list of enzymes not Type IIS.

In [48]:
a = Dseqrecord("GGTCTCatgccctaaccccccctaacccacGAGACC")
b = Dseqrecord("GGTCTCacccagtaaccatgtagtaaaaaaatgccGAGACC")

a1,a2,a3  = a.cut(BsaI)
b1,b2,b3 = b.cut(BsaI)

list_seqs = [a2,b2]

list_enz = [EcoRI, BamHI] # not type IIs

When trying to design the sequences with this list of enzymes, it occurs an error and shows an explanation why (No type IIs restriction enzymes available on this list that do not cut within the sequences).

In [51]:
gg_designer = GoldenGateDesigner(list_seqs, list_enz)

ValueError: No type IIs restriction enzymes available on this list
                        that do not cut within the sequences

### Example 4

Creating a list of sequences, some with sticky ends and some needing primers.

In [58]:
a = Dseqrecord("GGTCTCatgccctaaccccccctaacccacGAGACC")
b = Dseqrecord("GGTCTCacccagtaaccatgtagtaaaaaaatgccGAGACC")
c = Dseqrecord("GGTCTCtatgctcgggggggggggcagtacagtaGAGACC")

a1,a2,a3  = a.cut(BsaI)
b1,b2,b3 = b.cut(BsaI)
c1,c2,c3 = c.cut(BsaI)

e = primer_design(Dseqrecord("ccagttgctaacccttccttggtgttgaacaagatcgacgacatttcgttcgaaacttacgatg")) # AMPLICON
f = primer_design(Dseqrecord("ccagttgctaacccttccttggtcgtttcgttcgaaacttacgatg")) # AMPLICON
g = primer_design(Dseqrecord("tatgctcgggggggaacaagatcgacgacatttcgttcgaaacttacgagggggcagtacagta")) # AMPLICON

list_seqs = [a2, b2, c2, e, f, g]

list_enz = [EcoRI, BsaI, BsmBI, BamHI, BpmI]


With **GoldenGateDesigner** it is possible to see that sequences **e**, **f** and **g** need primers to be added while the first ones don't need changes.

In [61]:
gg_designer = GoldenGateDesigner(list_seqs, list_enz)
print(gg_designer)
print()

for seq in gg_designer[0]:
    print(seq.figure())
    print()

([Dseqrecord(-22), Dseqrecord(-27), Dseqrecord(-26), Dseqrecord(-86), Dseqrecord(-68), Dseqrecord(-86)], BsaI)

Dseqrecord(-22)
tgccctaaccccccctaa    
    gattgggggggattgggt

Dseqrecord(-27)
cccagtaaccatgtagtaaaaaa    
    cattggtacatcatttttttacg

Dseqrecord(-26)
atgctcgggggggggggcagta    
    agcccccccccccgtcatgtca

Dseqrecord(-86)
GGTCTCacagtccagttgctaacccttccttggtgttgaacaagatcgacgacatttcgttcgaaacttacgatgACTTtGAGACC
CCAGAGtgtcaggtcaacgattgggaaggaaccacaacttgttctagctgctgtaaagcaagctttgaatgctacTGAAaCTCTGG

Dseqrecord(-68)
GGTCTCaACTTccagttgctaacccttccttggtcgtttcgttcgaaacttacgatgAGTTtGAGACC
CCAGAGtTGAAggtcaacgattgggaaggaaccagcaaagcaagctttgaatgctacTCAAaCTCTGG

Dseqrecord(-86)
GGTCTCaAGTTtatgctcgggggggaacaagatcgacgacatttcgttcgaaacttacgagggggcagtacagtatgcctGAGACC
CCAGAGtTCAAatacgagcccccccttgttctagctgctgtaaagcaagctttgaatgctcccccgtcatgtcatacggaCTCTGG



After enzymatic restriction:

In [66]:
new_list = []

for i in gg_designer[0]:
    if i.cut(gg_designer[1]):
        i1, i2, i3 = i.cut(gg_designer[1])
        new_list.append(i2)
    else:
        new_list.append(i)


GoldenGateAssembler:

In [70]:
print('GoldenGateAssembler:')
gg_assembler = GoldenGateAssembler(new_list)
print(gg_assembler)

print()

print(gg_assembler[(6, 1, 2, 3, 4, 5)])


GoldenGateAssembler:
{(1, 2): Dseqrecord(-45), (1, 2, 3): Dseqrecord(-67), (1, 2, 3, 4): Dseqrecord(-135), (1, 2, 3, 4, 5): Dseqrecord(-185), (2, 3): Dseqrecord(-49), (2, 3, 4): Dseqrecord(-117), (2, 3, 4, 5): Dseqrecord(-167), (2, 3, 4, 5, 6): Dseqrecord(-235), (3, 4, 5, 6, 1): Dseqrecord(-230), (3, 4): Dseqrecord(-94), (3, 4, 5): Dseqrecord(-144), (3, 4, 5, 6): Dseqrecord(-212), (4, 5, 6, 1): Dseqrecord(-208), (4, 5, 6, 1, 2): Dseqrecord(-231), (4, 5): Dseqrecord(-122), (4, 5, 6): Dseqrecord(-190), (5, 6, 1): Dseqrecord(-140), (5, 6, 1, 2): Dseqrecord(-163), (5, 6, 1, 2, 3): Dseqrecord(-185), (5, 6): Dseqrecord(-122), (6, 1): Dseqrecord(-90), (6, 1, 2): Dseqrecord(-113), (6, 1, 2, 3): Dseqrecord(-135), (6, 1, 2, 3, 4): Dseqrecord(-203), (6, 1, 2, 3, 4, 5): Dseqrecord(o249)}

Dseqrecord
circular: True
size: 249
ID: id
Name: name
Description: description
Number of features: 18
/molecule_type=DNA
Dseq(o249)
AGTT..gatg
TCAA..ctac
